In [3]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/myenv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/myenv/bin/python3'

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/13 22:10:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/13 22:10:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<SparkContext master=local[2] appName=Create-DataFrame>

In [5]:
sc = spark.sparkContext
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")
print(type(pan_tadeusz_file))

**Zadanie 1**  

Wykorzystując plik z treścią Pana Tadeusza zaprezentowany w przykładach policz ile jest linii, w których zawiera się słowo `Tadeusz`. Wykorzystaj akcje i transformacje RDD.

In [6]:
# Filtruj linie, które zawierają słowo 'Tadeusz'
tadeusz_lines = pan_tadeusz_file.filter(lambda line: 'Tadeusz' in line)
# Policz, ile jest takich linii
tadeusz_count = tadeusz_lines.count()
print(f"Liczba linii zawierających 'Tadeusz': {tadeusz_count}")

[Stage 0:>                                                          (0 + 2) / 2]

Liczba linii zawierających 'Tadeusz': 182


**Zadanie 2**  
Wykorzystując metodę `top()` dla obiektu RDD wyświetl 3 najdłuższe linie wczytane z pliku z książką Pan Tadeusz.

In [7]:
# Użyj top() do wyświetlenia trzech najdłuższych linii
top_longest_lines = pan_tadeusz_file.top(3, key=lambda line: len(line))
print("3 najdłuższe linie:")
for line in top_longest_lines:
    print(line)

3 najdłuższe linie:
O niebezpieczeństwach wynikających z nieporządnego obozowania — Odsiecz niespodziana — Smutne położenie szlachty — Odwiedziny kwestarskie są wróżbą ratunku — Major Płut zbytnią zalotnością ściąga na siebie burzę — Wystrzał z krócicy, hasło boju — Czyny Kropiciela, czyny i niebezpieczeństwa Maćka — Konewka zasadzką ocala Soplicowo — Posiłki jezdne, atak na piechotę — Czyny Tadeusza — Pojedynek dowódców przerwany zdradą — Wojski stanowczym manewrem przechyla szalę boju — Czyny krwawe Gerwazego — Podkomorzy zwycięzca wspaniałomyślny.
Plany myśliwskie Telimeny — Ogrodniczka wybiera się na wielki świat i słucha nauk opiekunki — Strzelcy wracają — Wielkie zadziwienie Tadeusza — Spotkanie się powtórne w Świątyni dumania i zgoda ułatwiona za pośrednictwem mrówek — U stołu wytacza się rzecz o łowach — Powieść Wojskiego o Rejtanie i księciu Denassów, przerwana — Zagajenie układów między stronami, także przerwane — Zjawisko z kluczem — Kłótnia — Hrabia z Gerwazym odbywają radę

**Zadanie 3**  
Wykorzystując listę stopwords z adresu https://github.com/bieli/stopwords/blob/master/polish.stopwords.txt wykorzystaj akcje i transformacje RDD i wygeneruj listę unikalnych słów z pliku z treścią Pana Tadeusza (sprawdź transformację countByValue()) pomijając powyższe słowa stop oraz wszelkie znaki przestankowe. Wynik zapisz do słownika, a następnie do pliku json o nazwie pan_tadeusz_bag_of_words.json.
Które słowo występuje w tym tekście najczęściej? Wyświetl je z wyników wygenerowanych powyżej.

In [11]:
import re
import json
import requests

stopwords_url = "https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt"
stopwords = set(requests.get(stopwords_url).text.splitlines())
# stopwords

In [14]:
def clean_line(line):
    return re.sub(r'[^a-zA-ZąćęłńóśźżĄĆĘŁŃÓŚŹŻ ]', '', line.lower()).split()

# Przekształcenia na RDD
words_rdd = (
    pan_tadeusz_file
    .flatMap(clean_line)
    .filter(lambda word: word not in stopwords and word)
)
words_rdd.take(10)

['adam',
 'mickiewicz',
 'tadeusz',
 'ostatni',
 'zajazd',
 'litwie',
 'isbn',
 'księga',
 'pierwsza',
 'gospodarstwo']

In [17]:
word_counts = words_rdd.countByValue()
# word_counts

In [19]:
bag_of_words = dict(word_counts)
# bag_of_words

In [20]:
with open("pan_tadeusz_bag_of_words.json", "w", encoding="utf-8") as f:
    json.dump(bag_of_words, f, ensure_ascii=False, indent=4)

most_common_word = max(bag_of_words, key=bag_of_words.get)
print(f"Najczęściej występujące słowo: '{most_common_word}' - {bag_of_words[most_common_word]} razy")

Najczęściej występujące słowo: 'rzekł' - 155 razy
